In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from state import get_state, queue
from basic.reward_basic import calculate_reward
from basic.learner_basic import Learner
from traffic_lights_maps import get_traffic_light_phases, make_map
import os
import sys

import traci
import random
import gym
from gym import spaces

In [19]:
from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
import numpy as np
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [29]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [31]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [32]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [33]:
sumoBinary = "/Users/arseneclaustre/sumo/bin/sumo-gui"
sumoBinary2 = "/Users/arseneclaustre/sumo/bin/sumo"
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
# sumoConfig3 = r"/home/psels/code/psels/RL_traffic/RL_traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=10000)


sumoCmd = [sumoBinary2, "-c", sumoConfig3, "--start"]
sumoCmd = [sumoBinary2, "-c", sumoConfig3, "--start", "--no-warnings"]

for episode in range(300):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(10000): ## TO CHANGED
        epsilon = max(1 - episode / 300, 0.01)

        if step%1000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state = np.array(get_state(lane_ids))
            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))

            list_values.append(queue(lane_ids))
            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            #print("action", action)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


episode : 0
Step #100.00 (1ms ~= 10.00*RT, ~5000.00UPS, TraCI: 0ms, vehicles TOT 33 ACT 5 BUF 0)       
 Retrying in 1 seconds


episode : 1
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 11 BUF 0)                    
 Retrying in 1 seconds


episode : 2
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 2 BUF 0)                     
 Retrying in 1 seconds


episode : 3
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 8 BUF 0)                     
 Retrying in 1 seconds


...

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.6333605  -0.772122   -0.66759074 -0.7398742 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.4758344 -1.5594916 -1.4659417 -1.646508 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 6 7 4 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-3.270423  -3.2604744 -3.1265404 -3.5092452]
state : [0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 3 4 6 0 0 0 4 4 1 2
 0 0 0 0 0 0 0 0 1 2 0]
Q_values : [-2.4054735 -2.0149539 -1.073045  -0.3977217]
state : [0 1 1 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 4 1 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 4 0 1 0]
Q_values : [-1.2616742 -1.4178619  1.1073315 -0.733378 ]
state : [0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 4 3
 0 0 0 2 0 3 0 5 0 4 0]
Q_values : [0.15931252 0.19173327 0.13270363 1.883233  ]
state : [0 1

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.66068745 -0.772826   -0.7090702  -0.7682253 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.4645166 -1.5400261 -1.4961536 -1.6147523]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 6 7 4 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-3.0130746 -2.8930292 -2.5845098 -2.539297 ]
state : [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 6 4 0 0 0 4 4 1 2
 0 0 0 0 0 0 1 1 1 0 0]
Q_values : [-3.220171  -2.8951297 -1.2034413 -1.3813727]
state : [0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 4 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 2 0 5 0]
Q_values : [-2.3684602  -1.9811825  -0.9159379   0.36272296]
state : [0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 4 1 3
 0 2 0 0 0 3 0 4 0 5 0]
Q_values : [0.15584616 0.06505413 1.5003666  0.25155783]
state : 

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.6272886  -0.8418608  -0.67780083 -0.74853903]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.4009384 -1.6122108 -1.441912  -1.5775965]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 6 7 4 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.8964367 -2.9418285 -2.4626498 -2.4007344]
state : [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 6 4 0 0 0 4 4 1 2
 0 0 0 0 0 0 1 1 1 0 0]
Q_values : [-2.8950858  -2.7838674  -0.92410266 -1.1785386 ]
state : [0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 4 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 2 0 5 0]
Q_values : [-1.9261588  -1.8288039  -0.7257606   0.53916395]
state : [0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 4 1 3
 0 2 0 0 0 3 0 4 0 5 0]
Q_values : [0.40237373 0.19568524 1.5899445  0.30355334]
stat

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.6522219 -0.8633042 -0.7736078 -0.7999909]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.4963593 -1.6540062 -1.5972743 -1.6695001]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 6 7 4 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-3.0949745 -3.0219507 -2.6335342 -2.4486046]
state : [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 6 4 0 0 0 4 4 1 2
 0 0 0 0 0 0 1 1 1 0 0]
Q_values : [-3.0419796 -2.8263597 -1.0745864 -1.2060711]
state : [0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 4 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 2 0 5 0]
Q_values : [-1.9278998 -1.8037462 -0.7876275  0.5533175]
state : [0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 4 1 3
 0 2 0 0 0 3 0 4 0 5 0]
Q_values : [0.30942377 0.19937429 1.6787121  0.31428176]
state : [0 0 1 0

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.607703   -0.7421094  -0.6715033  -0.74461806]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.4326668 -1.4950455 -1.4711963 -1.6069936]
state : [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 6 4 0 0 0 4 4 1 2
 0 0 0 0 0 0 1 1 1 0 0]
Q_values : [-3.2352278 -2.8648696 -1.2710934 -1.3486413]
state : [0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 4 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 2 0 5 0]
Q_values : [-2.2293286  -1.9356282  -1.1044686   0.32019153]
state : [0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 4 1 3
 0 2 0 0 0 3 0 4 0 5 0]
Q_values : [ 0.00158164 -0.03301627  1.4068341   0.28087077]
state : [0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 3 0 4 0 3 0 3 0 0 0]
Q_values : [1.0019219 0.8842121 0.5727254 1.3191638]
state : 

In [37]:
sumoBinary = "/Users/arseneclaustre/sumo/bin/sumo-gui"
sumoBinary2 = "/Users/arseneclaustre/sumo/bin/sumo"
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
# sumoConfig3 = r"/home/psels/code/psels/RL_traffic/RL_traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)


sumoCmd = [sumoBinary, "-c", sumoConfig3, "--start"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(10000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%500 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        # action = np.random.randint(8)
        print("action", action)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.5403265  -0.713232   -0.6213437  -0.67982894]
action 0
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 17.522911 -17.96099   11.951095  15.162727]
action 0
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.3854567 -1.491927  -1.4199526 -1.6172345]
action 0
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 4 5 3 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.8333791 -1.9001443 -1.8013786 -2.0373268]
action 4
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 6 7 4 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-3.2402422 -3.2289848 -3.1345963 -3.5542994]
action 4
state : [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 5 5 6 0 0 0 3 1 0 1
 0 0 0 0 0 0 0 0 1 1 0]
Q

FatalTraCIError: Connection closed by SUMO.